In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/02 21:51:56 WARN Utils: Your hostname, Hema resolves to a loopback address: 127.0.1.1; using 172.30.190.161 instead (on interface eth0)
24/03/02 21:51:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/02 21:51:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/02 21:51:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

In [ ]:
!gunzip -c fhvhv_tripdata_2021-01.csv.gz > fhvhv_tripdata_2021.csv

In [ ]:
!wc -l fhvhv_tripdata_2021.csv

In [ ]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021.csv')

In [ ]:
df.show()

In [ ]:
df.head(5) # All values are parsed strings

In [ ]:
df.schema

In [ ]:
!head -n 1001 fhvhv_tripdata_2021.csv > head.csv

In [ ]:
!wc -l head.csv

In [3]:
import pandas as pd

In [4]:
print(pd.__version__)

1.5.3


In [5]:
df_pandas = pd.read_csv('head.csv')

In [6]:
df_pandas.head(5)

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
0,HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,NaN
1,HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,NaN
2,HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,NaN
3,HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,NaN
4,HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,NaN


In [7]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [8]:
spark.createDataFrame(df_pandas).show()

/home/ubuntu/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ubuntu/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|    NaN|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|    NaN|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|    NaN|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|    NaN|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|    NaN|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [9]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [10]:
from pyspark.sql import types

In [12]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True), 
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropoff_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True)
])

In [14]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021.csv')

In [15]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [16]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [17]:
df = df.repartition(24) # lazy load. Will not execute any jobs

In [18]:
df.write.parquet('fhvhv/2021/01/')

In [19]:
df = spark.read.parquet('fhvhv/2021/01/')

In [20]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True)])

In [21]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [22]:
# SELECT * FROM df WHERE hvfhs_license_num = HV0003

In [ ]:
from spark.sql import functions as F # Includes both system and user defined sql functions

In [23]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-02 20:11:50|2021-01-02 20:19:46|          76|         222|   null|
|           HV0005|              B02510|2021-01-07 08:28:45|2021-01-07 08:39:55|         224|         140|   null|
|           HV0003|              B02764|2021-01-03 11:10:58|2021-01-03 11:18:51|          49|          25|   null|
|           HV0003|              B02875|2021-01-06 18:44:53|2021-01-06 19:08:25|         144|         198|   null|
|           HV0003|              B02764|2021-01-02 15:19:48|2021-01-02 15:53:36|         259|         126|   null|
|           HV0003|              B02835|2021-01-02 18:14:24|2021-01-02 18:33:48|

In [24]:
# Create a user defined function which converts a field to a hexa decimal format
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [25]:
crazy_stuff('B02884')

's/b44'

In [27]:
from pyspark.sql import functions as F

In [28]:
# Define the user defined function to the list of available functions
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [29]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') 

DataFrame[base_id: string, pickup_date: date, dropoff_date: date, PULocationID: int, DOLocationID: int]

In [33]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-02 20:11:50|2021-01-02 20:19:46|          76|         222|   null|
|           HV0005|              B02510|2021-01-07 08:28:45|2021-01-07 08:39:55|         224|         140|   null|
|           HV0003|              B02764|2021-01-03 11:10:58|2021-01-03 11:18:51|          49|          25|   null|
|           HV0003|              B02875|2021-01-06 18:44:53|2021-01-06 19:08:25|         144|         198|   null|
|           HV0003|              B02764|2021-01-02 15:19:48|2021-01-02 15:53:36|         259|         126|   null|
|           HV0003|              B02835|2021-01-02 18:14:24|2021-01-02 18:33:48|

In [31]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-02|  2021-01-02|          76|         222|
|  e/9ce| 2021-01-07|  2021-01-07|         224|         140|
|  e/acc| 2021-01-03|  2021-01-03|          49|          25|
|  e/b3b| 2021-01-06|  2021-01-06|         144|         198|
|  e/acc| 2021-01-02|  2021-01-02|         259|         126|
|  s/b13| 2021-01-02|  2021-01-02|         148|         186|
|  e/acc| 2021-01-01|  2021-01-01|         216|         216|
|  e/9ce| 2021-01-04|  2021-01-04|          50|          80|
|  s/acd| 2021-01-04|  2021-01-04|         163|          90|
|  e/b3e| 2021-01-05|  2021-01-05|          69|          78|
|  s/b3d| 2021-01-02|  2021-01-02|          94|          18|
|  e/acc| 2021-01-06|  2021-01-06|          20|         119|
|  e/9ce| 2021-01-06|  2021-01-06|          82|         260|
|  e/acc| 2021-01-03|  2

In [36]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')

DataFrame[pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int]

In [35]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003').show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-03 11:10:58|2021-01-03 11:18:51|          49|          25|
|2021-01-06 18:44:53|2021-01-06 19:08:25|         144|         198|
|2021-01-02 15:19:48|2021-01-02 15:53:36|         259|         126|
|2021-01-02 18:14:24|2021-01-02 18:33:48|         148|         186|
|2021-01-01 04:01:37|2021-01-01 04:07:20|         216|         216|
|2021-01-04 19:01:45|2021-01-04 19:10:24|         163|          90|
|2021-01-05 19:45:32|2021-01-05 19:56:43|          69|          78|
|2021-01-02 10:55:03|2021-01-02 11:02:21|          94|          18|
|2021-01-06 12:50:58|2021-01-06 13:21:46|          20|         119|
|2021-01-03 03:03:18|2021-01-03 04:06:58|         216|         265|
|2021-01-06 16:29:53|2021-01-06 16:53:03|         146|          79|
|2021-01-02 04:28:39|2021-01-02 04:37:31|       

In [34]:
!head -n 10 head.csv

hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,
HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,
HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,
HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,
HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,
HV0005,B02510,2021-01-01 00:06:59,2021-01-01 00:43:01,88,42,
HV0005,B02510,2021-01-01 00:50:00,2021-01-01 01:04:57,42,151,
HV0003,B02764,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226,
HV0003,B02875,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255,
